# Accessing UK Model Surface data from Microsoft Planetary Computer

Set-up the pystac client to access the Microsoft Planetary Computer catalog

In [1]:
from pystac_client import Client
import planetary_computer

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

Define collection and assets to retrieve and construct [STAC API filters](https://github.com/stac-api-extensions/filter) for efficient query performance against Planetary Computer API

In [ ]:
collections = ["met-office-uk-deterministic-near-surface"]
asset_id = "temperature_at_surface"
forecast_extension_filters = {
    "op": "and",
    "args": [
        {
            "op": "=",
            "args": [
                {"property": "forecast:reference_datetime"},
                "2026-01-21T09:00:00Z",
            ],
        },
        {"op": "=", "args": [{"property": "forecast:horizon"}, "PT0054H00M"]},
    ],
}

Search Planetary Computer catalog for STAC items and retrieve STAC Asset URL

In [ ]:
search = catalog.search(
    collections=collections,
    filter_lang="cql2-json",
    filter=forecast_extension_filters,
)

asset_url = next(search.items()).assets[asset_id].href

StopIteration: 

Example usage: Plot NetCDF data on a map

In [ ]:
import fsspec
import xarray as xr
import matplotlib.pyplot as plt

example_netcdf = xr.open_dataset(fsspec.open(asset_url, expand=True).open())
plt.figure(figsize=(10, 5))
example_netcdf["surface_temperature"].plot()